# 介紹 tf.placeholder, session 可以一次 run 多個 op 的方法
When I read "TensorFlow in a Nutshell" series article (Ynote: "TensorFlow in a Nutshell — Part One Two Three–Camron's Blog") I feel the need to review Morvan's tf. So I do it now but with jupyter notebook.

In [1]:
import peforth

In [2]:
%%f if playing in jupyter notebook 
\ Now we redefine the 'unknown' command that does nothing at default  
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.
: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;
code # print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 


reDef unknown


In [3]:
"""
Know more, visit my Python tutorial page: https://morvanzhou.github.io/tutorials/
My Youtube Channel: https://www.youtube.com/user/MorvanZhou

Dependencies:
tensorflow: 1.1.0
"""
import tensorflow as tf

In [4]:
x1 = tf.placeholder(dtype=tf.float32, shape=None, name='one')
# name 只能這時候給, 然後就是 read-only 了. default 是 'placeholder:0', placeholder_1, _2...etc

In [5]:
%f x1 tib.

x1 tib. \ ==> Tensor("one:0", dtype=float32) (<class 'tensorflow.python.framework.ops.Tensor'>)


In [6]:
y1 = tf.placeholder(dtype=tf.float32, shape=None, name='two')
%f y1 tib.

y1 tib. \ ==> Tensor("two:0", dtype=float32) (<class 'tensorflow.python.framework.ops.Tensor'>)


In [7]:
z1 = x1 + y1  # 這個 + 應該是 tf.add() 從 z1.name 是 'add:0' 可見得, 不如試試看? --> 果然!
z11 = tf.add(x1, y1)  # 看看產生的東西跟上一行一不一樣? --> [x] z11.op 我看是一樣的
%f z1 tib.
%f z11 tib.

z1 tib. \ ==> Tensor("add:0", dtype=float32) (<class 'tensorflow.python.framework.ops.Tensor'>)
z11 tib. \ ==> Tensor("Add:0", dtype=float32) (<class 'tensorflow.python.framework.ops.Tensor'>)


In [8]:
# 上面 z1 是一個 op, 另做一個不相干的 op z2 以下示範執行時可以一次 sess.run() 好幾個 op  
x2 = tf.placeholder(dtype=tf.float32, shape=[2, 1])
y2 = tf.placeholder(dtype=tf.float32, shape=[1, 2])
z2 = tf.matmul(x2, y2)
%f x2 tib.
%f y2 tib.
%f z2 tib.

x2 tib. \ ==> Tensor("Placeholder:0", shape=(2, 1), dtype=float32) (<class 'tensorflow.python.framework.ops.Tensor'>)
y2 tib. \ ==> Tensor("Placeholder_1:0", shape=(1, 2), dtype=float32) (<class 'tensorflow.python.framework.ops.Tensor'>)
z2 tib. \ ==> Tensor("MatMul:0", shape=(2, 2), dtype=float32) (<class 'tensorflow.python.framework.ops.Tensor'>)


In [15]:
with tf.Session() as sess:
    # when only one operation to run
    z1_value = sess.run(z1, feed_dict={x1: 1, y1: 2})

    # when run multiple operations
    z1_value, z2_value = sess.run(
        [z1, z2],       # run them together
        feed_dict={
            x1: 1, y1: 2,
            x2: [[2], [2]], y2: [[3, 3]]
        })
    # print(z1_value)
    # print(z2_value)
%f z1_value tib.
%f z2_value tib.

z1_value tib. \ ==> 3.0 (<class 'numpy.float32'>)
z2_value tib. \ ==> [[6. 6.]
 [6. 6.]] (<class 'numpy.ndarray'>)
